<a href="https://colab.research.google.com/github/sebswU/cf-python/blob/master/botTrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is the training for the nlp model


Credit for instructions on tokenizing/preprocessing/neural network planning:\
Marco Vasquez @ Kaggle\
Tim @ Youtube\
Auréilin Géron @ O'Reily Media\
Kylie Ying @ Freecodecamp.org\
tensorflow.org website\
FARES SAYAH @ Kaggle\
Hayawi K, Shahriar S, Serhani MA, Taleb I, Mathew SS. ANTi-Vax: a novel Twitter dataset for COVID-19 vaccine misinformation detection. Public Health. 2022;203:23-30. doi:10.1016/j.puhe.2021.11.022\
Samikshya Siwakoti, Kamya Yadav, Isra Thange, Nicola Bariletto, Luca Zanotti, Alaa Ghoneim, and Jacob N. Shapiro. Localized Misinformation in a Global Pandemic: Report on COVID-19 Narratives around the World. Empirical Study of Conflict, Princeton University, pages 1-68, March 2021. URL https://esoc.princeton.edu/publications/localized-misinformation-global-pandemic-report-covid-19-narratives-around-world. \



Credit for datasets:\
Poynter.org
Steven @ Kaggle
Princeton



In [2]:
%pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.9/189.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 45.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Succ

import modules for preprocessing and model development

In [3]:
from google.colab import drive
drive.mount('/content/drive/')
!pip -q install transformers
import os
os.chdir('/content/drive/My Drive')
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow_hub as hub
import sklearn as sk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.5 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


read and clean/preprocess data

1. load data from google drive

In [4]:
os.chdir('/content/drive/My Drive/archive (1)')
import pandas as pd
databox = pd.DataFrame(pd.read_csv('news_articles.csv'))
more_false_data = pd.DataFrame(pd.read_csv('poynter_data.csv'))
esocData = pd.DataFrame(pd.read_csv('esoc.csv'))
nlmData = pd.DataFrame(pd.read_csv('ids.csv'))
trueorfalsenlm = pd.DataFrame(pd.read_csv('VaxMisinfoData.csv'))

In [5]:
trueorfalsenlm

,id,is_misinfo
0,1344795424855642112,0
1,1344794858133860353,0
2,1344794822691983360,0
3,1344794752819077123,1
4,1344792070507134977,0
...,...,...
15068,1413087751474397186,0
15069,1413087030578401283,0
15070,1413085793397186565,1
15071,1413085519710363648,0


some data and labels are separate datasets so this will concatenate them together

In [6]:
trueorfalsenlm = trueorfalsenlm.assign(text=nlmData['text'])


See how they are all structured


2. Clean data


*   tweet id was not needed in nlm dataset
*   the name label will be used in all datasets to represent the column for label (0 as real, 1 as fake, due to the goal of detecting false data)
*   nlm dataset was messed up after a certain point, was trimmed
*   databox dataset had string labels, convert to number
*   databox dataset will be added by instance to train, so delete all irrelevant
*   rename databox column to standardize all data column names to 'text'
*   esoc data was also messed up after instance 6000 so it was trimmed too
*   more_false_data needed a label column so a Pandas Series of 1.0s were added
*   esoc data has irrelevant columns, those were deleted
*   esoc data based on researching TYPES of misinfo, so all instances are false; labels are all 1.0s
*   rename esoc text column to 'text' to standardize for the merging



In [7]:
trueorfalsenlm = trueorfalsenlm.drop(axis = 1, columns = ['id'])
trueorfalsenlm['is_misinfo'] = trueorfalsenlm['is_misinfo'].astype(float)
trueorfalsenlm.rename(columns = {'is_misinfo':'label'}, inplace = True)
trueorfalsenlm = trueorfalsenlm[:12381]

In [8]:
databox['label'].replace ({'Fake': 1, 'Real': 0}, inplace=True)
databox = databox.drop(axis = 1, columns = ['author','published','title','text','language','site_url','main_img_url','type','title_without_stopwords','hasImage'])
databox.rename(columns = {'text_without_stopwords':'text'}, inplace = True)

databox.head()


,label,text
0,0.0,print pay back money plus interest entire fami...
1,0.0,attorney general loretta lynch plead fifth bar...
2,0.0,red state fox news sunday reported morning ant...
3,0.0,email kayla mueller prisoner tortured isis cha...
4,0.0,email healthcare reform make america great sin...


In [9]:
more_false_data.head()
more_false_data['label'] = float(1)
more_false_data

,label,text
0,1.0,A post that has a list of recommendations to ...
1,1.0,The first person to received a Chinese COVID-...
2,1.0,A video from family doctor Natalia Prego Canc...
3,1.0,RNA-based COVID-19 vaccines can turn humans i...
4,1.0,A post that shows a doctor from Santiago del ...
...,...,...
7539,1.0,The coronavirus was created in a lab and pate...
7540,1.0,A Chinese market caused the new coronavirus (...
7541,1.0,The peak of the new coronavirus will happen i...
7542,1.0,Stores and supermarkets in Veracruz (Mexico) ...


In [10]:
esocData = esocData[:5613]
esocData = esocData.assign(label=float(1))
newPD = pd.DataFrame()
esocData['Primary_Language']

0          English
1          English
2          English
3          English
4          English
           ...    
5608    Portuguese
5609    Portuguese
5610    Portuguese
5611    Portuguese
5612    Portuguese
Name: Primary_Language, Length: 5613, dtype: object

In [11]:
esocData.drop(axis = 1, columns = ['s_no','Notes','Region','Reported_On','Additional_Reporting','Retrieve_from_1','Retrieve_from_2','Retrieve_from_3','Twitter_Reference','Direct_Post_1','Direct_Post_2','Direct_Post_3','Direct_Post_4','Title','Publication_Date','Entry_Date','Primary_Country','Secondary_Country','Primary_Language','Secondary_Language','Main_Narrative','Recoded_Main_Narrative','Recoded_Narrative_Coder','ChainMsg_or_Email','Motive','Motive_Description','Source','Source_Description','Distrib_Channel','Misinfo_Type','Key_Words','Summary','Coder'], inplace=True)
esocData.rename(columns = {'Narrative_Description':'text'}, inplace = True)

In [12]:
#esocData = esocData.iloc[:, [1,0]]
esocData

,text,label
0,Hindi language YouTube account saying COVID-19...,1.0
1,Twitter user posting a compilation video of pe...,1.0
2,Twitter user posting a video of a celebrity ea...,1.0
3,Chain message of NYPD containment responses.,1.0
4,Chain message of Indian Health Ministry respon...,1.0
...,...,...
5608,Coffee can prevent COVID-19,1.0
5609,The number of COVID-19 deaths is 946,1.0
5610,Facemasks donated by China contain the virus,1.0
5611,low-quality masks were distributed by the Braz...,1.0


concatenate the dataset before applying the address remover because addresses and handles are common on multiple datasets

In [13]:
data = pd.concat([databox, esocData, trueorfalsenlm, more_false_data])
data

,label,text
0,0.0,print pay back money plus interest entire fami...
1,0.0,attorney general loretta lynch plead fifth bar...
2,0.0,red state fox news sunday reported morning ant...
3,0.0,email kayla mueller prisoner tortured isis cha...
4,0.0,email healthcare reform make america great sin...
...,...,...
7539,1.0,The coronavirus was created in a lab and pate...
7540,1.0,A Chinese market caused the new coronavirus (...
7541,1.0,The peak of the new coronavirus will happen i...
7542,1.0,Stores and supermarkets in Veracruz (Mexico) ...


remove @, 'photo by name/Getty images',dates, links

In [14]:
import re
import string

#TODO: strip the dataset of all emails, twitter acc prof, image references, capital letters
web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
user = re.compile(r"(?i)@[a-z0-9_]+")
regex_list = [web_address, user]
# we then use the sub method to replace anything matching

def clean(dataset):
  """clean from stopwords and other stuff, convert to list, back to string"""
  if isinstance(dataset, float):
    dataset = 'a'
  STOPWORDS = stopwords.words('english')
    # Check characters to see if they are in punctuation
  nopunc = [char for char in dataset if char not in string.punctuation and all(regex.search(char.lower()) is None for regex in regex_list)]

    # Join the characters again to form the string.
  nopunc = ''.join(nopunc)

    # Now just remove any stopwords
  return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])
  return " ".join(word
                for word in nopunc.split()
                if word.lower() not in stopwords.words("english") and all(regex.search(word.lower()) is None for regex in regex_list))

In [15]:
esocData['text'] = esocData['text'].apply(clean)
trueorfalsenlm['text'] = trueorfalsenlm['text'].apply(clean)
more_false_data['text'] = more_false_data['text'].apply(clean)

In [16]:
esocData[esocData['label']!=0.0]

,text,label
0,Hindi language YouTube account saying COVID19 ...,1.0
1,Twitter user posting compilation video people ...,1.0
2,Twitter user posting video celebrity eating ba...,1.0
3,Chain message NYPD containment responses,1.0
4,Chain message Indian Health Ministry responses,1.0
...,...,...
5608,Coffee prevent COVID19,1.0
5609,number COVID19 deaths 946,1.0
5610,Facemasks donated China contain virus,1.0
5611,lowquality masks distributed Brazilian government,1.0


In [17]:
data = pd.concat([esocData, trueorfalsenlm, more_false_data, databox], sort = False, ignore_index = True)
data = data.iloc[:27584]
data = data.fillna(0)
data['label'] = data['label'].astype(int)
data = data[['label','text']]

In [18]:
data[data['label'] == 1]

,label,text
0,1,Hindi language YouTube account saying COVID19 ...
1,1,Twitter user posting compilation video people ...
2,1,Twitter user posting video celebrity eating ba...
3,1,Chain message NYPD containment responses
4,1,Chain message Indian Health Ministry responses
...,...,...
27526,1,democrats perpetually circle wagons republican...
27527,1,election crossroads socialism capitalism exclu...
27528,1,print applaud integrity condemn selling allow ...
27529,1,new country women minorities hit hardest ann c...


In [19]:
import numpy as np
train, val, test = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])

In [20]:
train.text

21551    photo shows dozens coffins allegedly sale coro...
15018    can’t believe personally know idiots won’t tak...
24443    Shanghai government officially recommends Vita...
23389                 single Muslim died coronavirus China
3228     Claims 14 25 confirmed cases COVID19 announced...
                               ...                        
12280    get vaccine still wear mask social distance pe...
12340     birthday near corner thing want FRIGGING VACCINE
22801    video unprecedented stone rain Rome suggests f...
18347    Gov Gavin Newsom “has legal authority” make we...
9718     Oh goodness could cry dad booked covid vaccine...
Name: text, Length: 16550, dtype: object

In [21]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  """returns a PrefetchDataset object with instances truncated to batch_size"""
  df = dataframe.copy()
  labels = df.pop('label')
  df = df['text']
  ds = tf.data.Dataset.from_tensor_slices((df, labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(tf.data.AUTOTUNE)
  return ds

In [22]:
train

,label,text
21551,1,photo shows dozens coffins allegedly sale coro...
15018,0,can’t believe personally know idiots won’t tak...
24443,1,Shanghai government officially recommends Vita...
23389,1,single Muslim died coronavirus China
3228,1,Claims 14 25 confirmed cases COVID19 announced...
...,...,...
12280,0,get vaccine still wear mask social distance pe...
12340,0,birthday near corner thing want FRIGGING VACCINE
22801,1,video unprecedented stone rain Rome suggests f...
18347,1,Gov Gavin Newsom “has legal authority” make we...


In [23]:
train = df_to_dataset(train)
val = df_to_dataset(val)
test = df_to_dataset(test)

In [24]:
train

<_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

streamline the text, make all lowercase

checking dataset structure

split data into training and testing

count # of unique words

:The embeddings can be used as a piece to build a larger machine learning system that is able to draw insights about COVID-19 research.
last touch: the input layer has a max sequence of 100 bc discord messages rarely have more than 100 words/tokens (anecdotal)

In [25]:
import tensorflow as tf
import tensorflow_hub as hub
#english_sentences = tf.constant(["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."])
"""
this is the reference for the encoder and the embedder
the preprocessor converts the words into an array that is fed to the model
the encoder takes it and converts it to unique # representation (like oneHot)
the embedding vectorizes the word-number combination in a way that emphasizes
context (think about dots on a 3d graph and those dots' location represented
context in regards to each other)
model will be trained on data, and embeddings will be fitted to accomodate
after saved, specialized embedding/encoding/trained model will process new data
outside of set

from stkovf: LSTM layers expect 3D input tensors, but Dense outputs only 2D
fix:
"""

text_input = tf.keras.layers.Input(shape=(100,), dtype=tf.string)

embed = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2",
                       input_shape = [], dtype=tf.string, trainable=True)

In [26]:
list(train)[0][0]

<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'photo shows Japanese medical professional providing aid Wuhan',
       b'United States rejects South Korean Covid19 inaccurate',
       b'COVID19 Breeds Rapidly Toilet Paper',
       b'Claims US money paid March April rent repaid',
       b'Facebook user suggests confirmed cases suspected cases COVID19 whereas declared positive people actually novel coronavirus',
       b'ever touched nyc subway pole don\xe2\x80\x99t worry what\xe2\x80\x99s vaccine',
       b'cant wait Celebritys finally find conned deadly vaccine promoting Im going Piss laughing amp Jack Daniels well \xf0\x9f\xa4\xa3\xf0\x9f\xa4\xa3\xf0\x9f\xa4\xa3\xf0\x9f\xa4\xa3\xf0\x9f\xa4\xa3',
       b'published oct daily sheeple joe joseph quickly discusses new system put place detroit international airport sixteen airports nationwide classic problem reaction solution make incredibly miserable travel people gladly give rights away convenience',
       b'everybody send drudge

In [30]:
checkpoint_fname = "training/ckpt_01.ckpt"
ceckpoint_dir = os.path.dirname(checkpoint_fname)

callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_fname,
                                          save_weights_only=True,
                                          verbose=1)



In [31]:
model = tf.keras.Sequential()
model.add(embed)
model.add(tf.keras.layers.Dense(8, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(8, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

model.compile(optimizer='adamax',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [32]:
model.evaluate(train)

518/518 [==============================] - 7s 4ms/step - loss: 0.7163 - accuracy: 0.3503


[0.7162803411483765, 0.3503323197364807]

In [33]:
model.evaluate(val)

173/173 [==============================] - 1s 4ms/step - loss: 0.7162 - accuracy: 0.3565


[0.716232419013977, 0.35653436183929443]

In [34]:
hist = model.fit(train, epochs = 10, validation_data = val,callbacks=[callback])

Epoch 1/10
518/518 [==============================] - ETA: 0s - loss: 0.6305 - accuracy: 0.6645
Epoch 1: saving model to training/ckpt_01.ckpt
518/518 [==============================] - 134s 254ms/step - loss: 0.6305 - accuracy: 0.6645 - val_loss: 0.5557 - val_accuracy: 0.6998
Epoch 2/10
518/518 [==============================] - ETA: 0s - loss: 0.5237 - accuracy: 0.7244
Epoch 2: saving model to training/ckpt_01.ckpt
518/518 [==============================] - 88s 169ms/step - loss: 0.5237 - accuracy: 0.7244 - val_loss: 0.4554 - val_accuracy: 0.7484
Epoch 3/10
518/518 [==============================] - ETA: 0s - loss: 0.4594 - accuracy: 0.7622
Epoch 3: saving model to training/ckpt_01.ckpt
518/518 [==============================] - 67s 130ms/step - loss: 0.4594 - accuracy: 0.7622 - val_loss: 0.4167 - val_accuracy: 0.7622
Epoch 4/10
518/518 [==============================] - ETA: 0s - loss: 0.4267 - accuracy: 0.7735
Epoch 4: saving model to training/ckpt_01.ckpt
518/518 [================

In [37]:
os.listdir(ceckpoint_dir)

['ckpt_01.ckpt.data-00000-of-00001', 'ckpt_01.ckpt.index', 'checkpoint']

In [ ]:
vectorizer = tf.keras.layers.TextVectorization(max_tokens = 2000)
vectorizer.adapt(train.map(lambda txt, label: txt))

In [ ]:
modeLSTM = tf.keras.Sequential([
  vectorizer,
  tf.keras.layers.Embedding(
      input_dim = len(vectorizer.get_vocabulary()),
      output_dim = 12,
      mask_zero = True
  ),
  tf.keras.layers.LSTM(12),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(1, activation = 'sigmoid')

])
modeLSTM.compile(optimizer=tf.keras.optimizers.Adamax(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"
),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
#pytorch implementation of the above code
#(GENERATED FROM CHATGPT FOR FUN, DID NOT CREATE BY HAND)
import torch
import torch.nn as nn

class ModelLSTM(nn.Module):
    def __init__(self, input_dim, output_dim, embedding_dim, hidden_dim):
        super(ModelLSTM, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=0, mask_zero=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense1 = nn.Linear(hidden_dim, 10)
        self.dropout = nn.Dropout(p=0.4)
        self.dense2 = nn.Linear(10, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x, (hidden, _) = self.lstm(x)
        x = self.dense1(hidden.squeeze(0))
        x = self.dropout(x)
        x = self.dense2(x)
        return x

# Create an instance of the model
model = ModelLSTM(input_dim=len(vectorizer.get_vocabulary()),
                  output_dim=1,
                  embedding_dim=12,
                  hidden_dim=12)

# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-7)

In [ ]:


history = modeLSTM.fit(train, epochs=10, validation_data = val)

Epoch 1/10
518/518 [==============================] - 80s 140ms/step - loss: 0.0000e+00 - accuracy: 0.3008 - val_loss: 0.0000e+00 - val_accuracy: 0.3061
Epoch 2/10
518/518 [==============================] - 41s 79ms/step - loss: 0.0000e+00 - accuracy: 0.3004 - val_loss: 0.0000e+00 - val_accuracy: 0.3061
Epoch 3/10
518/518 [==============================] - 27s 53ms/step - loss: 0.0000e+00 - accuracy: 0.3004 - val_loss: 0.0000e+00 - val_accuracy: 0.3061
Epoch 4/10
518/518 [==============================] - 20s 38ms/step - loss: 0.0000e+00 - accuracy: 0.3004 - val_loss: 0.0000e+00 - val_accuracy: 0.3061
Epoch 5/10
486/518 [===========================>..] - ETA: 1s - loss: 0.0000e+00 - accuracy: 0.2998

In [ ]:
modeLSTM.evaluate(test)

173/173 [==============================] - 2s 9ms/step - loss: 0.4369 - accuracy: 0.7834


[0.43692976236343384, 0.7833967804908752]

The model actually works to an extent. Granted, the results are the inverse of the label predictions, but a general pattern is detected. There are some anomalies, like the score given to 'covid was created in a lab', but since some of the data date back to early in the pandemic and was conducted before more research was done on the virus, speculation and uncertainty of truth is expected. In addition, the model seems to skew more sensitive towards political messages rather than misinformation purely based on the charasteristics of the disease. The current tests have stopwords directly fed into the model, so perhaps that might tamper with the weights and biases that were tuned on the preprocessed dataset.

In [ ]:
model.predict(['bioweapon'])

1/1 [==============================] - 0s 73ms/step


array([[0.35603696]], dtype=float32)

In [ ]:
model.predict(['pelosi'])

1/1 [==============================] - 0s 27ms/step


array([[0.74125236]], dtype=float32)

In [ ]:
model.predict(['it is recommended that you get tested for covid if flu-like symptoms occur'])

1/1 [==============================] - 0s 33ms/step


array([[0.966005]], dtype=float32)

In [ ]:
model.predict(['i got vaccinated so it should be safe'])

1/1 [==============================] - 0s 27ms/step


array([[0.4906899]], dtype=float32)

In [ ]:
model.predict(['beach ball'])

1/1 [==============================] - 0s 29ms/step


array([[0.8127688]], dtype=float32)

In [ ]:
model.predict(['covid was created in a lab'])

1/1 [==============================] - 0s 29ms/step


array([[0.99525416]], dtype=float32)

In [ ]:
model.predict(['covid'])

1/1 [==============================] - 0s 29ms/step


array([[0.85981816]], dtype=float32)

In [ ]:
model.predict(['the vaccine is killing babies'])

1/1 [==============================] - 0s 27ms/step


array([[0.4099576]], dtype=float32)

In [ ]:
model.predict(['i think covid will give you cancer but im not sure'])

1/1 [==============================] - 0s 26ms/step


array([[0.4162552]], dtype=float32)

In [ ]:
model.predict(['yes covid will give you cancer'])

1/1 [==============================] - 0s 25ms/step


array([[0.5954191]], dtype=float32)

In [ ]:
model.predict(['the radical left is planning to decrease the older population with the vaccine'])

1/1 [==============================] - 0s 28ms/step


array([[0.5560514]], dtype=float32)

In [ ]:
model.predict(['covid fake propaganda populist theory vaccine bad'])

1/1 [==============================] - 0s 30ms/step


array([[0.34776455]], dtype=float32)

In [ ]:
import tempfile
MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

export_path = /tmp/1



In [ ]:
!ls -l {export_path}

total 180
drwxr-xr-x 2 root root   4096 Dec 26 05:59 assets
-rw-r--r-- 1 root root  10744 Dec 26 05:59 keras_metadata.pb
-rw-r--r-- 1 root root 159919 Dec 26 05:59 saved_model.pb
drwxr-xr-x 2 root root   4096 Dec 26 05:59 variables


In [ ]:
saved_model_cli show --dir {export_path} --all

SyntaxError: ignored

In [ ]:
tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
    )

In [ ]:
modeLSTM.save('modeLSTM')
#import tensorflowjs as tfjs

#checkpoint_path = "./checkpoints/cp-{epoch:05d}.ckpt"
#modeLSTM.save_weights(checkpoint_dir.format(epoch=0))
#tfjs.converters.save_keras_model(model, '/content/drive/My Drive/archive (1)')
#modeLSTM.save('/content/drive/My Drive/archive (1)', save_format = 'h5')

In [ ]:
%tensorflowjs_converter --input_format keras \ modelstuff/model.h5 \ modelstuff

UsageError: Line magic function `%tensorflowjs_converter` not found.


In [ ]:
import torch
from torch import nn

In [ ]:
num_input_features=784
num_output_features=10


class MLP(nn.module):
  def __init__(self, num_input_features, num_output_features):
    super(MLP, self).__init__()
    self.fc1=nn.Linear(num_input_features, 128)
    self.fc2=nn.Linear(128,64)
    self.fc3=nn.Linear(64, num_output_features)

  def forward(self,x):
    x = self.fc1(x)
    x = torch.relu(x)
    x = self.fc2(x)
    x = torch.relu(x)
    x = self.fc3(x)
    #no relu required bc it gives the final output
    return x

model = MLP(num_input_features, num_output_features)

model.train()

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters,lr=0.01)


